# Topic Modeling using LDA

### References

* Data: Drug Dataset (400EA)
* Preprocess: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
* LDA: https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/09/lda/

### Load Raw Data

In [61]:
import article_db_conn
import pandas as pd
import re

pd.set_option('display.max_colwidth', 999)

In [62]:
article_titles = article_db_conn.select_query("SELECT table_title FROM article_tables")
columns = ['table_title']
article_titles_pd = pd.DataFrame(article_titles, columns=columns).reset_index()
article_titles_pd.head()

,index,table_title
0,0,"Table 2 Clinical, Doppler Echocardiographic and Hemodynamic Characteristics in the Patients Grouped According to Baseline MFP and Acute Change After Loading Manipulations"
1,1,"Table 3 Association (Multivariate Logistic Regression Analysis) Between Either Baseline MFP or Its Acute Change After Loading Manipulations and Tolerance to Beta-Blocker Treatment, Adjusting for Gender, Etiology, NYHA Class, LVEF and Peak VO2"
2,2,Table 1 Baseline and Clinical Characteristics of the Two Patient Groups
3,3,Table 2 Clinical Course of the Two Patient Groups
4,4,Table 3 Characteristics and In-Hospital Outcome of the Study and the Registry Patients


In [63]:
regex = re.compile(r'Table [0-9]* ')

In [64]:
article_titles_pd.table_title = article_titles_pd.table_title.apply(lambda x: x.replace(regex.search(x).group(), '') if regex.search(x) != None else x)

In [65]:
len(article_titles)

20000

In [66]:
article_titles_pd.head()

,index,table_title
0,0,"Clinical, Doppler Echocardiographic and Hemodynamic Characteristics in the Patients Grouped According to Baseline MFP and Acute Change After Loading Manipulations"
1,1,"Association (Multivariate Logistic Regression Analysis) Between Either Baseline MFP or Its Acute Change After Loading Manipulations and Tolerance to Beta-Blocker Treatment, Adjusting for Gender, Etiology, NYHA Class, LVEF and Peak VO2"
2,2,Baseline and Clinical Characteristics of the Two Patient Groups
3,3,Clinical Course of the Two Patient Groups
4,4,Characteristics and In-Hospital Outcome of the Study and the Registry Patients


### Preprocessing

In [67]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gracelee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

* Preprocess
 1. simple_preprocess: Split Text by whitespace
 2. STOPWORDS: Remove stopwords
 3. lemmatize_stemming
 
* lemmatize_stemming
 - Lemmatizing & Stemming Replace word with original form
 - Lemmatizing consider whether the word exist in the real world
 - pos means a position of the word
 - https://m.blog.naver.com/PostView.nhn?blogId=vangarang&logNo=220963244354&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [68]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

* Run

In [69]:
%time processed_docs = article_titles_pd['table_title'].map(preprocess)
processed_docs[:10]

CPU times: user 14.5 s, sys: 31.1 ms, total: 14.6 s
Wall time: 14.6 s


0                                                                                                                          [clinic, doppler, hemodynam, characterist, patient, group, accord, baselin, acut, chang, load, manipul]
1                                                            [associ, multivari, logist, regress, analysi, baselin, acut, chang, load, manipul, toler, beta, blocker, treatment, adjust, gender, etiolog, nyha, class, lvef, peak]
2                                                                                                                                                                                  [baselin, clinic, characterist, patient, group]
3                                                                                                                                                                                                  [clinic, cours, patient, group]
4                                                                                           

----

### T-SNE

* https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/
* https://lumiamitie.github.io/r/python/tsne-for-r-py/

In [70]:
### TSNE모델에는 transform 메소드가 없고 fit_transform만 있음
# library import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

np.random.seed(2018)

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
%time vect.fit([' '.join(d) for d in processed_docs])
%time tsne_data = vect.transform([' '.join(d) for d in processed_docs]).toarray()

CPU times: user 582 ms, sys: 12.7 ms, total: 594 ms
Wall time: 594 ms
CPU times: user 651 ms, sys: 319 ms, total: 970 ms
Wall time: 974 ms


In [73]:
tsne_data[:10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
%time tsne_result = TSNE(learning_rate=300, init='pca').fit_transform(np.array(tsne_data))

In [ ]:
tsne_result[:10]

In [ ]:
# # 시각화
# plt.scatter(tsne_result[:, 1], tsne_result[:, 0])
# plt.xlim(tsne_result[:, 1].min()-3, tsne_result[:, 1].max()+3) # 최소, 최대
# plt.ylim(tsne_result[:, 0].min()-3, tsne_result[:, 0].max()+3) # 최소, 최대
# plt.xlabel('t-SNE 특성0') # x축 이름
# plt.ylabel('t-SNE 특성1') # y축 이름
# plt.show() # 그래프 출력

In [ ]:
with open('./drug_db_tsne_result.pkl', 'wb') as f:
    pickle.dump(tsne_result, f)

In [ ]:
%time tsne_3d_result = TSNE(n_components=3, learning_rate=300, init='pca').fit_transform(np.array(tsne_data))

In [ ]:
tsne_3d_result[:10]

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D

# plt.style.use('fivethirtyeight')

# plt.rcParams["figure.figsize"] = (20,10)
# plt.rcParams['lines.linewidth'] = 1
# plt.rcParams['lines.color'] = 'r'
# plt.rcParams['axes.grid'] = True 

# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')

# for x, y, z in tsne_3d_result:
#     ax.scatter(x, y, z, c='blue')
    
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

----

### LDA

* Setting Variables

    1. document_topic_counts : List of Counter (len = count of documents)
    2. topic_word_counts : List of Counter (len = count of topic)
    3. topic_counts : List of Integer (len = count of topic)
    4. document_lengths : List of length of documents
    5. distinct_words: All unique words in dataset
    6. V: length of distinct words
    7. D: length of documents
    
* Counter Object
 - Calculate count of elements

In [ ]:
from collections import Counter

def get_variables(K):
    # 사용자가 원하는 토픽의 갯수
    K = 8

    # 각 토픽이 각 문서에 할당되는 횟수
    # Counter로 구성된 리스트
    # 각 Counter는 각 문서를 의미
    document_topic_counts = [Counter() for _ in processed_docs]

    # 각 단어가 각 토픽에 할당되는 횟수
    # Counter로 구성된 리스트
    # 각 Counter는 각 토픽을 의미
    topic_word_counts = [Counter() for _ in range(K)]

    # 각 토픽에 할당되는 총 단어수
    # 숫자로 구성된 리스트
    # 각각의 숫자는 각 토픽을 의미함
    topic_counts = [0 for _ in range(K)]

    # 각 문서에 포함되는 총 단어수
    # 숫자로 구성된 리스트
    # 각각의 숫자는 각 문서를 의미함
    document_lengths = list(map(len, processed_docs))

    # 단어 종류의 수
    distinct_words = set(word for document in processed_docs for word in document)
    V = len(distinct_words)

    # 총 문서의 수
    D = len(processed_docs)

    return V, D, document_topic_counts, topic_word_counts, topic_counts, document_lengths, distinct_words

In [ ]:
def p_topic_given_document(topic, d, alpha=0.1):
    # 문서 d의 모든 단어 가운데 topic에 속하는
    # 단어의 비율 (alpha를 더해 smoothing)
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word, topic, beta=0.1):
    # topic에 속한 단어 가운데 word의 비율
    # (beta를 더해 smoothing)
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

def topic_weight(d, word, k):
    # 문서와 문서의 단어가 주어지면
    # k번째 토픽의 weight를 반환
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

In [ ]:
def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

import random
def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i

* Run
 - Initialize Topic using random value by word in documents
 - Calculate variables
    1. document_topic_counts
        - count of topic word in every document
        - 개별 문서에서 topic word의 등장 횟수
    2. topic_word_counts
        - appearance count of words in whole documents
        - every word seperate by topic
        - 개별 Topic에서 topic word의 등장 횟수(전체 문서 기준)

In [ ]:
random.seed(0)

K = 8
V, D, document_topic_counts, topic_word_counts, topic_counts, document_lengths, distinct_words = get_variables(K)

# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document] for document in processed_docs]

# 위와 같이 랜덤 초기화한 상태에서 
# AB를 구하는 데 필요한 숫자를 세어봄
for d in range(D):
    for word, topic in zip(processed_docs[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [ ]:
len(processed_docs)

----

In [ ]:
import time
start_time = time.time() 

for iter in range(3000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(processed_docs[d], document_topics[d])):
            # 깁스 샘플링 수행을 위해
            # 샘플링 대상 word와 topic을 제외하고 세어봄
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # 깁스 샘플링 대상 word와 topic을 제외한 
            # 말뭉치 모든 word의 topic 정보를 토대로
            # 샘플링 대상 word의 새로운 topic을 선택
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # 샘플링 대상 word의 새로운 topic을 반영해 
            # 말뭉치 정보 업데이트
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1
    
    if iter % 500 == 0:
        print("--- %d iter: %s mins ---" % (iter, str((time.time() - start_time) / 60.)))

print("--- %s mins ---" % str((time.time() - start_time) / 60.))

In [ ]:
import operator

doc_result = documents[['index', 'Origin_Text']]
doc_result.columns = ['id', 'document']
doc_result['topic'] = doc_result.id.apply(lambda x: max(document_topic_counts[x].items(), key=operator.itemgetter(1))[0])
doc_result['topic_prob'] = doc_result.id.apply(lambda x: max(document_topic_counts[x].items(), key=operator.itemgetter(1))[1])
doc_result['topic_word'] = doc_result.topic.apply(lambda x: ','.join(['%s(%s)' % (a, b)for a, b in topic_word_counts[x].most_common(10)]))
doc_result = pd.merge(doc_result, pd.DataFrame(tsne_result, columns=['plot_x', 'plot_y']), left_index=True, right_index=True)
doc_result = pd.merge(doc_result, pd.DataFrame(tsne_3d_result, columns=['td_x', 'td_y', 'td_z']), left_index=True, right_index=True)

In [ ]:
plt.style.use('fivethirtyeight')

plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.color'] = 'r'
plt.rcParams['axes.grid'] = True 

# doc_result.plot.scatter(x='plot_x', y='plot_y', c='topic', colormap='Accent')

In [ ]:
# threedee = plt.figure().gca(projection='3d')
# threedee.scatter(doc_result.td_x, doc_result.td_y, doc_result.td_z, c=doc_result.topic)

# plt.savefig('3d_scatter_lda.png')

* 깁스 샘플링(Gibbs Sampling) 
    * http://4four.us/article/2014/10/lda-parameter-estimation
    * https://bab2min.tistory.com/569

* PyLDAvis
    * https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/

----

In [ ]:
import pyLDAvis.gensim

In [ ]:
# numpy.ndarray, shape = (n_topics, n_terms)
topic_term_dists = np.array([topic_word_counts[i][k] for i in range(K) for k in list(distinct_words)]).reshape((K, len(distinct_words))) 

# numpy.ndarray, shape = (n_docs, n_topics)
doc_topic_dists = pd.DataFrame([d.values() for d in document_topic_counts]).fillna(0).values

# numpy.ndarray, shape = (n_docs,)
doc_lengths = np.array(document_lengths)

# list of str, vocab list
vocab = list(distinct_words)

# numpy.ndarray, shape = (n_vocabs,)
term_frequency = np.array([topic_word_counts[i][k] for i in range(K) for k in list(distinct_words)]).reshape((K, len(distinct_words))).sum(axis=0)

* topic_term_dists: topic_term_dists
* doc_topic_dists: doc_topic_dists
* doc_lengths: doc_lengths
* vocab: vocab
* term_frequency: term_frequency

In [ ]:
lda_mallet_data = {
    'topic_term_dists':topic_term_dists,
    'doc_topic_dists':doc_topic_dists,
    'doc_lengths':doc_lengths,
    'vocab':vocab,
    'term_frequency':term_frequency
}
vis_data = pyLDAvis.prepare(**lda_mallet_data)
# pyLDAvis.display(vis_data)
# pyLDAvis.save_html(vis_data, 'test.html')

In [38]:
# LDAvis의 우측 HBar Chart Data
# Freq: Estimated term frequency within the selected topic
# Total: Overall term frequency
print(vis_data.topic_info.Category.unique())
vis_data.topic_info[vis_data.topic_info.Category == 'Topic1'].sort_values('Freq', ascending=False).head()

['Default' 'Topic1' 'Topic2' 'Topic3' 'Topic4' 'Topic5' 'Topic6' 'Topic7'
 'Topic8']


,Category,Freq,Term,Total,loglift,logprob
term,,,,,,
291,Topic1,503910.0,regress,503910.0,8.4280,3.4965
656,Topic1,473370.0,model,548574.0,8.2806,3.4340
749,Topic1,366480.0,hazard,426575.0,8.2762,3.1781
1185,Topic1,351210.0,multivari,361067.0,8.4004,3.1355
768,Topic1,351210.0,analysi,638765.0,7.8299,3.1355


----

## Visualization

### 1. Main View
* Layout: https://www.codingfactory.net/10530

#### a. HBar Chart
* Data: vis_data.topic_info[vis_data.topic_info.Category == 'Topic1'].sort_values('Freq', ascending=False).head()
* D3: http://bl.ocks.org/erikvullings/51cc5332439939f1f292

In [39]:
import json

hbar_json = {}
hbar_json['labels'] = vis_data.topic_info.Category.unique().tolist()
hbar_json['max_width'] = vis_data.topic_info[vis_data.topic_info.Category != 'Default'][['Total']].max()[0]
for l in vis_data.topic_info.Category.unique().tolist():
    tmp_df = vis_data.topic_info[vis_data.topic_info.Category == l].sort_values(['Category', 'Freq'], ascending=[True, False]).groupby('Category').head()
    sub_json = {}

    hbar_json[l] = list(tmp_df[['Term', 'Freq', 'Total']].sort_values('Freq', ascending=False).reset_index().to_dict('index').values())
    
f = open('./Visualization/res/lda/hbar_data.json', 'w')
f.write(json.dumps(hbar_json, indent=4))
f.close()

#### b. Scatter Chart
* Data: tsne_result
* D3: https://bl.ocks.org/Niekes/1c15016ae5b5f11508f92852057136b5

In [40]:
doc_result = documents[['index', 'Origin_Text']]
doc_result.columns = ['id', 'document']
doc_result['topic'] = doc_result.id.apply(lambda x: max(document_topic_counts[x].items(), key=operator.itemgetter(1))[0])
doc_result['topic_prob'] = doc_result.id.apply(lambda x: max(document_topic_counts[x].items(), key=operator.itemgetter(1))[1])
doc_result['topic_word'] = doc_result.topic.apply(lambda x: ','.join(['%s(%s)' % (a, b)for a, b in topic_word_counts[x].most_common(10)]))
doc_result = pd.merge(doc_result, pd.DataFrame(tsne_result, columns=['plot_x', 'plot_y']), left_index=True, right_index=True)
doc_result = pd.merge(doc_result, pd.DataFrame(tsne_3d_result, columns=['td_x', 'td_y', 'td_z']), left_index=True, right_index=True)

/Users/gracelee/.pyenv/versions/for_jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gracelee/.pyenv/versions/for_jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/gracelee/.pyenv/versions/for_jupyter/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to

In [41]:
scatter_json = list(doc_result[['id', 'plot_x', 'plot_y', 'topic']].to_dict('index').values())

f = open('./Visualization/res/lda/scatter_data.json', 'w')
f.write(json.dumps(scatter_json, indent=4))
f.close()

#### c. Table
* Data: doc_result[['topic', 'document']].head()

In [42]:
doc_result.to_csv('./data_output/lda.tsv', sep='\t', index_label=False)

In [43]:
doc_result.groupby('topic').head(1)[['topic', 'topic_word']]

,topic,topic_word
0,0,"regress(33),model(31),hazard(24),ventricular(23),analysi(23),multivari(23),death(21),valu(21),proport(21),mortal(20)"
31,5,"coronari(47),arteri(17),patient(16),wall(15),detect(15),angiographi(13),flow(13),diagnost(13),segment(12),comput(12)"
50,1,"intervent(19),modern(13),women(12),method(12),estim(12),incom(11),level(11),high(9),countri(9),mean(8)"
51,2,"cancer(27),year(25),health(15),incid(14),develop(13),surviv(13),ratio(13),standardis(13),type(12),countri(12)"
66,6,"event(78),advers(70),treatment(66),grade(30),group(29),popul(28),patient(26),safeti(22),emerg(22),relat(17)"
69,3,"risk(49),score(37),stroke(27),factor(19),associ(19),patient(17),vasc(14),ischaem(14),accord(13),studi(12)"
150,7,"patient(57),outcom(24),hospit(22),acut(21),clinic(21),legend(20),myocardi(18),infarct(17),therapi(13),versus(11)"
195,4,"year(36),death(34),caus(33),rat(19),efficaci(19),specif(17),mortal(16),relat(16),adjust(16),chang(13)"


In [44]:
doc_result.groupby('topic').agg({'id': 'unique'})

,id
topic,
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 183, 184, 220, 240, 262, 263, 264, 266, 268, 270, 272, 274, 276, 278, 286, 288, 290]"
1,"[50, 52, 53, 54, 55, 60, 62, 64, 65, 68, 70, 71, 72, 76, 77, 79, 80, 81, 83, 85, 86, 87, 88, 89, 91, 93, 94, 96, 98, 99, 255, 366, 394]"
2,"[51, 56, 57, 58, 59, 61, 63, 67, 73, 74, 84, 200, 203, 204, 205, 206, 208, 212, 213, 215, 218, 221, 223, 225, 226, 231, 233, 236, 244, 248, 284, 292, 386]"
3,"[69, 163, 251, 256, 280, 282, 283, 289, 296, 297, 298, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 367, 368, 369, 370, 371, 372, 373, 374, 376, 377, 378, 379, 380, 381, 382, 383, 384, 387, 388, 389, 390, 391, 392, 393, 395, 396, 397, 398, 399]"
4,"[195, 201, 202, 207, 209, 210, 211, 214, 216, 217, 219, 222, 224, 227, 228, 229, 230, 232, 234, 235, 237, 238, 239, 241, 242, 243, 245, 246, 247, 249, 257, 260, 265, 269, 279, 287, 291, 293, 294]"
5,"[31, 37, 250, 253, 258, 261, 277, 285, 300, 303, 304, 305, 306, 307, 308, 309, 310, 313, 314, 315, 316, 317, 319, 320, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 385]"
6,"[66, 75, 78, 82, 90, 92, 95, 97, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 254, 271, 273, 299, 301, 302, 318]"
7,"[150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 252, 259, 267, 275, 281, 295, 311, 312, 321, 375]"
